In [1]:
!pip install xarray netCDF4 folium branca geopandas rasterio rioxarray pyproj shapely matplotlib seaborn pandas numpy dask

In [ ]:
import os
import numpy as np
import pandas as pd
import xarray as xr
import matplotlib.pyplot as plt
from scipy.spatial import cKDTree
from pandas.tseries.frequencies import to_offset


In [3]:
# ----------------------------
# 입력 파일 경로
# ----------------------------
OCEAN_CSV = "../Dataset/ocean_grid_points_0p25deg.csv"          # dataset 1
WIND10_NC = "../Dataset/netcdf4_korea_10m_wind.nc"             # dataset 2 (u10, v10)
WIND100_NC = "../Dataset/netcdf4_korea_100m_wind.nc"           # dataset 2 (u100, v100)
WAVE_NC = "../Dataset/data_stream-wave_stepType-instant.nc"    # dataset 3 (rhoao)

In [4]:
# ----------------------------
# 출력 폴더
# ----------------------------
OUT_BASE = "../output_maps"

FOLDER_MAP = {
    "Daily":   "Daily",
    "Monthly": "Monthly",
    "2M":      "2_Month",
    "3M":      "3_Month",
    "6M":      "6_Month",
    "Yearly":  "Yearly",
}

# 집계 사양
AGG_SPECS = [
    ("Daily",   "1D"),
    ("Monthly", "MS"),
    ("2M",      "2MS"),
    ("3M",      "3MS"),
    ("6M",      "6MS"),
    ("Yearly",  "YS"),
]

# 분석 기간 (2024년 전체)
T0, T1 = "2024-01-01T00:00:00", "2024-12-31T23:00:00"

# 옵션
MAKE_10M = True
OCEAN_MAX_DIST_DEG = 0.15

# Dask chunk 크기(시간축)
TIME_CHUNK = 168  # 1주 단위 정도

In [5]:
# ----------------------------
# 해양 마스크 생성 (dataset1 기반 KDTree)
# ----------------------------
def build_ocean_mask(lat_1d: np.ndarray, lon_1d: np.ndarray, ocean_df: pd.DataFrame,
                     max_dist_deg: float = 0.15) -> xr.DataArray:
    ocean_pts = np.column_stack([ocean_df["lat"].to_numpy(), ocean_df["lon"].to_numpy()])
    tree = cKDTree(ocean_pts)

    LAT, LON = np.meshgrid(lat_1d, lon_1d, indexing="ij")  # (lat, lon)
    target_pts = np.column_stack([LAT.ravel(), LON.ravel()])

    dist, _ = tree.query(target_pts, k=1)
    mask = (dist <= max_dist_deg).reshape(LAT.shape)

    return xr.DataArray(
        mask,
        coords={"latitude": lat_1d, "longitude": lon_1d},
        dims=("latitude", "longitude"),
        name="ocean_mask"
    )


In [6]:
# ----------------------------
# 풍속/풍향 계산
# ----------------------------
def wind_speed_dir_from(u: xr.DataArray, v: xr.DataArray):
    speed = np.hypot(u, v)
    # Meteorological "from" direction: 북=0°, 동=90° (바람이 불어오는 방향)
    wdir = (np.degrees(np.arctan2(-u, -v)) + 360.0) % 360.0
    speed = xr.DataArray(speed, coords=u.coords, dims=u.dims, name="wind_speed")
    wdir = xr.DataArray(wdir, coords=u.coords, dims=u.dims, name="wind_dir_from_deg")
    return speed, wdir

In [7]:
# ----------------------------
# 파일명 라벨: 시작~끝
# ----------------------------
def period_label(start_ts: pd.Timestamp, freq: str, hour_step: int = 1) -> str:
    off = to_offset(freq)
    end_ts = (start_ts + off) - pd.Timedelta(hours=hour_step)
    return f"{start_ts:%Y%m%d%H}_to_{end_ts:%Y%m%d%H}"


In [8]:
# ----------------------------
# 플롯 유틸 (범위 고정: vmin/vmax 사용)
# ----------------------------
def plot_speed_and_vectors(speed2d, u2d, v2d, title, out_png,
                           stride=3, vmin=0.0, vmax=None):
    lat = speed2d["latitude"].to_numpy()
    lon = speed2d["longitude"].to_numpy()
    LON, LAT = np.meshgrid(lon, lat)

    fig, ax = plt.subplots(figsize=(10, 7))
    pm = ax.pcolormesh(LON, LAT, speed2d.to_numpy(), shading="auto", vmin=vmin, vmax=vmax)
    cb = fig.colorbar(pm, ax=ax)
    cb.set_label("Wind speed (m/s)")

    ax.quiver(
        LON[::stride, ::stride], LAT[::stride, ::stride],
        u2d.to_numpy()[::stride, ::stride], v2d.to_numpy()[::stride, ::stride],
        scale=400, width=0.002
    )

    ax.set_title(title)
    ax.set_xlabel("Longitude")
    ax.set_ylabel("Latitude")
    ax.set_xlim(lon.min(), lon.max())
    ax.set_ylim(lat.min(), lat.max())
    ax.grid(True, linewidth=0.5, alpha=0.5)

    fig.tight_layout()
    fig.savefig(out_png, dpi=200)
    plt.close(fig)


def plot_direction_map(wdir2d, title, out_png):
    lat = wdir2d["latitude"].to_numpy()
    lon = wdir2d["longitude"].to_numpy()
    LON, LAT = np.meshgrid(lon, lat)

    fig, ax = plt.subplots(figsize=(10, 7))
    pm = ax.pcolormesh(LON, LAT, wdir2d.to_numpy(), shading="auto", vmin=0, vmax=360, cmap="hsv")
    cb = fig.colorbar(pm, ax=ax)
    cb.set_label("Wind direction (from, deg)")

    ax.set_title(title)
    ax.set_xlabel("Longitude")
    ax.set_ylabel("Latitude")
    ax.set_xlim(lon.min(), lon.max())
    ax.set_ylim(lat.min(), lat.max())
    ax.grid(True, linewidth=0.5, alpha=0.5)

    fig.tight_layout()
    fig.savefig(out_png, dpi=200)
    plt.close(fig)


def plot_power_density(power2d, title, out_png, vmin=0.0, vmax=None):
    lat = power2d["latitude"].to_numpy()
    lon = power2d["longitude"].to_numpy()
    LON, LAT = np.meshgrid(lon, lat)

    fig, ax = plt.subplots(figsize=(10, 7))
    pm = ax.pcolormesh(LON, LAT, power2d.to_numpy(), shading="auto", vmin=vmin, vmax=vmax)
    cb = fig.colorbar(pm, ax=ax)
    cb.set_label("Wind power density (W/m²)")

    ax.set_title(title)
    ax.set_xlabel("Longitude")
    ax.set_ylabel("Latitude")
    ax.set_xlim(lon.min(), lon.max())
    ax.set_ylim(lat.min(), lat.max())
    ax.grid(True, linewidth=0.5, alpha=0.5)

    fig.tight_layout()
    fig.savefig(out_png, dpi=200)
    plt.close(fig)

In [9]:
# ----------------------------
# 하위 폴더 생성: Speed/Direction/Power + 10m/100m
# ----------------------------
def ensure_output_dirs(base_time_dir: str):
    dirs = {}
    for var in ["Speed", "Direction", "Power"]:
        for h in ["10m", "100m"]:
            p = os.path.join(base_time_dir, var, h)
            os.makedirs(p, exist_ok=True)
            dirs[(var, h)] = p
    return dirs


In [10]:
# ----------------------------
# 한 주기(freq) 결과 생성
# - Speed/Power: 0~vmax 고정
# - vmax는 "해당 freq 전체 기간(예: Daily이면 연중 모든 일평균 결과)"에서의 최대값
# ----------------------------
def generate_maps_for_freq(
    freq_name: str,
    freq: str,
    ocean_mask: xr.DataArray,
    u10: xr.DataArray | None,
    v10: xr.DataArray | None,
    u100: xr.DataArray,
    v100: xr.DataArray,
    rho_on_wind_grid: xr.DataArray,
    out_time_dir: str,
):
    out_dirs = ensure_output_dirs(out_time_dir)

    # ---------- 100m: resample ----------
    spd100_inst = np.hypot(u100, v100)  # (time, lat, lon)
    u100_mean = u100.resample(valid_time=freq).mean()
    v100_mean = v100.resample(valid_time=freq).mean()
    spd100_mean = spd100_inst.resample(valid_time=freq).mean()

    power100_inst = 0.5 * rho_on_wind_grid * (spd100_inst ** 3)
    power100_mean = power100_inst.resample(valid_time=freq).mean()

    # vmax(100m): ocean only 기준
    vmax_speed_100 = float(spd100_mean.where(ocean_mask).max().compute().item())
    vmax_power_100 = float(power100_mean.where(ocean_mask).max().compute().item())

    # ---------- 10m: resample ----------
    if (u10 is not None) and (v10 is not None):
        spd10_inst = np.hypot(u10, v10)
        u10_mean = u10.resample(valid_time=freq).mean()
        v10_mean = v10.resample(valid_time=freq).mean()
        spd10_mean = spd10_inst.resample(valid_time=freq).mean()

        power10_inst = 0.5 * rho_on_wind_grid * (spd10_inst ** 3)
        power10_mean = power10_inst.resample(valid_time=freq).mean()

        vmax_speed_10 = float(spd10_mean.where(ocean_mask).max().compute().item())
        vmax_power_10 = float(power10_mean.where(ocean_mask).max().compute().item())
    else:
        u10_mean = v10_mean = spd10_mean = None
        power10_mean = None
        vmax_speed_10 = vmax_power_10 = None

    # ---------- 기간별 PNG 저장 ----------
    for t in u100_mean["valid_time"].to_numpy():
        ts = pd.Timestamp(t)
        label = period_label(ts, freq=freq, hour_step=1)

        # ===== 100m =====
        u2d = u100_mean.sel(valid_time=ts).where(ocean_mask)
        v2d = v100_mean.sel(valid_time=ts).where(ocean_mask)
        spd2d = spd100_mean.sel(valid_time=ts).where(ocean_mask)
        _, wdir2d = wind_speed_dir_from(u2d, v2d)
        p2d = power100_mean.sel(valid_time=ts).where(ocean_mask)

        plot_speed_and_vectors(
            speed2d=spd2d,
            u2d=u2d,
            v2d=v2d,
            title=f"[{freq_name}] Speed+Vectors (100m) | {label} | Ocean only",
            out_png=os.path.join(out_dirs[("Speed", "100m")], f"{label}_speed_vectors_100m.png"),
            stride=3,
            vmin=0.0,
            vmax=vmax_speed_100,
        )
        plot_direction_map(
            wdir2d=wdir2d,
            title=f"[{freq_name}] Direction(from) (100m) | {label} | Ocean only",
            out_png=os.path.join(out_dirs[("Direction", "100m")], f"{label}_direction_from_100m.png"),
        )
        plot_power_density(
            power2d=p2d,
            title=f"[{freq_name}] Power density (100m) | {label} | Ocean only",
            out_png=os.path.join(out_dirs[("Power", "100m")], f"{label}_power_density_100m.png"),
            vmin=0.0,
            vmax=vmax_power_100,
        )

        # ===== 10m (옵션) =====
        if spd10_mean is not None:
            u2d10 = u10_mean.sel(valid_time=ts).where(ocean_mask)
            v2d10 = v10_mean.sel(valid_time=ts).where(ocean_mask)
            spd2d10 = spd10_mean.sel(valid_time=ts).where(ocean_mask)
            _, wdir2d10 = wind_speed_dir_from(u2d10, v2d10)
            p2d10 = power10_mean.sel(valid_time=ts).where(ocean_mask)

            plot_speed_and_vectors(
                speed2d=spd2d10,
                u2d=u2d10,
                v2d=v2d10,
                title=f"[{freq_name}] Speed+Vectors (10m) | {label} | Ocean only",
                out_png=os.path.join(out_dirs[("Speed", "10m")], f"{label}_speed_vectors_10m.png"),
                stride=3,
                vmin=0.0,
                vmax=vmax_speed_10,
            )
            plot_direction_map(
                wdir2d=wdir2d10,
                title=f"[{freq_name}] Direction(from) (10m) | {label} | Ocean only",
                out_png=os.path.join(out_dirs[("Direction", "10m")], f"{label}_direction_from_10m.png"),
            )
            plot_power_density(
                power2d=p2d10,
                title=f"[{freq_name}] Power density (10m) | {label} | Ocean only",
                out_png=os.path.join(out_dirs[("Power", "10m")], f"{label}_power_density_10m.png"),
                vmin=0.0,
                vmax=vmax_power_10,
            )


In [11]:
def main():
    os.makedirs(OUT_BASE, exist_ok=True)

    # dataset1: ocean points
    ocean_df = pd.read_csv(OCEAN_CSV)  # columns: lat, lon

    # dataset2: wind (Dask chunks)
    ds10 = xr.open_dataset(WIND10_NC, chunks={"valid_time": TIME_CHUNK}).sortby(["latitude", "longitude"])
    ds100 = xr.open_dataset(WIND100_NC, chunks={"valid_time": TIME_CHUNK}).sortby(["latitude", "longitude"])

    # dataset3: wave rho (Dask chunks)
    ds_wave = xr.open_dataset(WAVE_NC, chunks={"valid_time": TIME_CHUNK}).sortby(["latitude", "longitude"])

    # 2024 slice
    u100 = ds100["u100"].sel(valid_time=slice(T0, T1))
    v100 = ds100["v100"].sel(valid_time=slice(T0, T1))

    if MAKE_10M:
        u10 = ds10["u10"].sel(valid_time=slice(T0, T1))
        v10 = ds10["v10"].sel(valid_time=slice(T0, T1))
    else:
        u10 = v10 = None

    # ocean mask on wind(0.25deg) grid
    lat = ds100["latitude"].to_numpy()
    lon = ds100["longitude"].to_numpy()
    ocean_mask = build_ocean_mask(lat, lon, ocean_df, max_dist_deg=OCEAN_MAX_DIST_DEG)

    # rho -> wind grid (공간 보간, 시간 유지)
    rho = ds_wave["rhoao"].sel(valid_time=slice(T0, T1))
    rho_on_wind_grid = rho.interp(latitude=ds100["latitude"], longitude=ds100["longitude"])

    # 주기별 생성
    for freq_name, freq in AGG_SPECS:
        time_dir = os.path.join(OUT_BASE, FOLDER_MAP[freq_name])
        os.makedirs(time_dir, exist_ok=True)

        generate_maps_for_freq(
            freq_name=freq_name,
            freq=freq,
            ocean_mask=ocean_mask,
            u10=u10,
            v10=v10,
            u100=u100,
            v100=v100,
            rho_on_wind_grid=rho_on_wind_grid,
            out_time_dir=time_dir,
        )

    print("All done. Output saved under:", OUT_BASE)


if __name__ == "__main__":
    main()


/tmp/ipykernel_16783/2693581942.py:8: UserWarning: The specified chunks separate the stored chunks along dimension "valid_time" starting at index 168. This could degrade performance. Instead, consider rechunking after loading.
  ds10 = xr.open_dataset(WIND10_NC, chunks={"valid_time": TIME_CHUNK}).sortby(["latitude", "longitude"])
/tmp/ipykernel_16783/2693581942.py:9: UserWarning: The specified chunks separate the stored chunks along dimension "valid_time" starting at index 168. This could degrade performance. Instead, consider rechunking after loading.
  ds100 = xr.open_dataset(WIND100_NC, chunks={"valid_time": TIME_CHUNK}).sortby(["latitude", "longitude"])
/tmp/ipykernel_16783/2693581942.py:12: UserWarning: The specified chunks separate the stored chunks along dimension "valid_time" starting at index 168. This could degrade performance. Instead, consider rechunking after loading.
  ds_wave = xr.open_dataset(WAVE_NC, chunks={"valid_time": TIME_CHUNK}).sortby(["latitude", "longitude"])


All done. Output saved under: ../output_maps
